#### Import Dependencies

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchaudio

import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os

#### Global Constants

In [2]:
load_dotenv()
DOWNLOAD_FOLDER = os.getenv('DOWNLOAD_FOLDER')

#### Util Functions / Classes

In [3]:
# Function to load an MP3 file and convert it to a mel spectrogram
def get_mel_spectrogram(filepath, sample_rate=22050, n_mels=128):
    waveform, sr = torchaudio.load(filepath)
    
    # Resample if necessary
    if sr != sample_rate:
        resampler = torchaudio.transforms.Resample(sr, sample_rate)
        waveform = resampler(waveform)
    
    # Convert to mel spectrogram
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=sample_rate,
        n_mels=n_mels
    )(waveform)
    
    # Normalize spectrogram
    mel_spectrogram = (mel_spectrogram - mel_spectrogram.mean()) / (mel_spectrogram.std() + 1e-6)
    
    return mel_spectrogram


In [4]:
# Dataset class to load data and process mel spectrograms
class SongDataset(Dataset):
    def __init__(self, csv_path, sample_rate=22050, n_mels=128, duration=30):
        self.data = pd.read_csv(csv_path)
        self.sample_rate = sample_rate
        self.n_mels = n_mels
        self.duration = duration
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Song filename follows this format: (index)^(video id)^(title).mp3 --> 0^LlWGt_84jpg^Special Breed.mp3 (example)
        song_filename = str(idx) + '^' + self.data.iloc[idx]['videoID'] + '^' + self.data.iloc[idx]['title'] + '.mp3'
        song_filepath = DOWNLOAD_FOLDER + '/' + song_filename
        viewcount = self.data.iloc[idx]['views']

        # Get the mel spectrogram
        mel_spectrogram = get_mel_spectrogram(song_filepath, self.sample_rate, self.n_mels)
        
        return mel_spectrogram, torch.tensor(viewcount, dtype=torch.float32)
    
    # Slice the dataset using lower and upper bounds 
    def slice(self, lower: int, upper: int):
        self.data = self.data.iloc[lower:upper]

#### Model Training

In [5]:
# Define the model architecture with CNN + RNN
class ViewCountPredictor(nn.Module):
    def __init__(self):
        super(ViewCountPredictor, self).__init__()
        
        # CNN for feature extraction
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2))
        )
        
        # RNN to handle variable-length sequences
        self.rnn = nn.LSTM(input_size=128, hidden_size=64, num_layers=2, batch_first=True)
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1)
        )
    
    def forward(self, x):
        # Extract features with CNN
        batch_size, channels, n_mels, time_steps = x.size()
        x = x.view(batch_size, 1, n_mels, time_steps)  # Ensure single channel dimension for CNN
        x = self.cnn(x)  # [batch_size, 128, new_n_mels, new_time_steps]
        
        # Flatten frequency dimension for RNN
        x = x.mean(dim=2)  # Global average pooling across frequency dimension
        x = x.permute(0, 2, 1)  # [batch_size, time_steps, 128]
        
        # Pass through RNN
        _, (hn, _) = self.rnn(x)
        x = hn[-1]  # Take the last hidden state
        
        # Fully connected layers
        x = self.fc(x)
        return x

In [6]:
def train_model(model, train_loader, val_loader, num_epochs=20, lr=1e-3):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for mel_spectrogram, viewcount in train_loader:
            mel_spectrogram = mel_spectrogram.unsqueeze(1)  # Add channel dimension
            viewcount = viewcount.unsqueeze(1)  # Match output shape
            
            optimizer.zero_grad()
            output = model(mel_spectrogram)
            loss = criterion(output, viewcount)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")
        
        # Validation Step
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for mel_spectrogram, viewcount in val_loader:
                mel_spectrogram = mel_spectrogram.unsqueeze(1)
                viewcount = viewcount.unsqueeze(1)
                output = model(mel_spectrogram)
                loss = criterion(output, viewcount)
                val_loss += loss.item()
        
        print(f"Validation Loss: {val_loss / len(val_loader)}")

#### Training Hyperparameters

In [7]:
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 1e-3

#### Main Code

In [8]:
# Initializing Dataset and DataLoader
dataset = SongDataset(csv_path='data/songs_data.csv')
dataset.slice(0, 10)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Initialize and train the model
model = ViewCountPredictor()
train_model(model, train_loader, test_loader, num_epochs=EPOCHS, lr=LEARNING_RATE)

/home/alean/anaconda3/envs/ess/lib/python3.10/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


RuntimeError: stack expects each tensor to be equal size, but got [2, 128, 58054] at entry 0 and [2, 128, 5131] at entry 1